In [2]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from pathlib import Path
import pandas as pd
import warnings
import requests
# import datetime ########## from datetime import datetime으로 변경
from datetime import datetime
# import optuna ########## 사용하지 않으므로 삭제
import json
import copy
import os

print('!!!!!!!!!!!!!!! Start park_analysis.py !!!!!!!!!!!!!!!')
path = Path.cwd().parent
raw_data = str(path) + '/raw_data/'
data = str(path) + '/data/'
result = str(path) + '/result_data/'

warnings.filterwarnings(action='ignore')

# 데이터 로드
print(datetime.now().strftime('%Y.%m.%d %Hh%Mm%Ss'), 'Load Data')
df = pd.read_csv(data+'dataset.csv')
predict_df = pd.read_csv(data+'predict_dataset.csv')

# train, test 나누기
train_data, test_data = train_test_split(df, test_size=0.33, random_state=42)

X_train, y_train = train_data.drop('혼잡도', axis=1), train_data['혼잡도']
X_test, y_test = test_data.drop('혼잡도', axis=1), test_data['혼잡도']

# 모델 적합
print(datetime.now().strftime('%Y.%m.%d %Hh%Mm%Ss'), 'Fit Model')
model = LGBMClassifier(random_state=42) ######## 실행할 때마다 결과가 바뀌면 안 됨 --> random_state 지정
model.fit(X_train, y_train, categorical_feature=['주차장'], eval_set=(X_test,y_test), verbose=1)

# 적합된 모델로 혼잡도 예측

!!!!!!!!!!!!!!! Start park_analysis.py !!!!!!!!!!!!!!!
2022.12.26 14h54m06s Load Data
2022.12.26 14h54m06s Fit Model
[1]	valid_0's multi_logloss: 0.871627
[2]	valid_0's multi_logloss: 0.789175
[3]	valid_0's multi_logloss: 0.725003
[4]	valid_0's multi_logloss: 0.67195
[5]	valid_0's multi_logloss: 0.628573
[6]	valid_0's multi_logloss: 0.591403
[7]	valid_0's multi_logloss: 0.559193
[8]	valid_0's multi_logloss: 0.53128
[9]	valid_0's multi_logloss: 0.506777
[10]	valid_0's multi_logloss: 0.486179
[11]	valid_0's multi_logloss: 0.468239
[12]	valid_0's multi_logloss: 0.452486
[13]	valid_0's multi_logloss: 0.438531
[14]	valid_0's multi_logloss: 0.426286
[15]	valid_0's multi_logloss: 0.414948
[16]	valid_0's multi_logloss: 0.404885
[17]	valid_0's multi_logloss: 0.395722
[18]	valid_0's multi_logloss: 0.387339
[19]	valid_0's multi_logloss: 0.379847
[20]	valid_0's multi_logloss: 0.372628
[21]	valid_0's multi_logloss: 0.365741
[22]	valid_0's multi_logloss: 0.359324
[23]	valid_0's multi_logloss: 0.3538

LGBMClassifier(random_state=42)

In [3]:
print(datetime.now().strftime('%Y.%m.%d %Hh%Mm%Ss'), 'Predict')
predict_df = predict_df[X_train.columns]

2022.12.26 14h54m18s Predict


In [6]:


predict = pd.DataFrame(model.predict(predict_df))
output = pd.concat([predict_df, predict], axis= 1)

In [ ]:
output = output.rename(columns={0:'혼잡도예측'})
output = output[['연','월','일','시','분','요일','주차장','혼잡도예측']]

# 예측일시, 시간대 추가
output['예측일시'] = output['연'].astype(str) + output['월'].astype(str) + output['일'].astype(str)
output['시간대'] = output['시'].astype(str) + output['분'].astype(str)

# 결과 테이블 만들기
result_table = pd.DataFrame(columns=['등록일시', '주차장명', '예측일시', '시간대', '혼잡도'])
result_table['주차장명'] = output['주차장']
result_table['예측일시'] = output['예측일시']
result_table['시간대'] = output['시간대']
result_table['혼잡도'] = output['혼잡도예측']
result_table['등록일시'] = datetime.now().strftime('%Y%m%d%H%M%S')
result_table = result_table.astype(str)
result_table.to_csv(result + 'park_analysis.csv', index=False, encoding='utf-8-sig')

#result_table.to_csv(result +'생존분석.csv', index = False, encoding = 'utf-8-sig')